In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
from astropy.constants import c
from astropy import constants as const
import astropy.units as u
from scipy import special


def weight(n):
    return 2 * n**2

def boltzmann(n1, n2, eV, T):
    k = 8.6173303e-5  # Boltzmann constant in eV/K
    g_n = weight(n2) / weight(n1) * np.exp(-eV / (k * T))
    return g_n

def CIV(T):
    sigma_Lya = 5.9e-14 * (T / 1e4) ** (-1/2)
    sigma_CIV = (0.19 / 0.4164) * sigma_Lya * (12) ** (0.5)
    N_CIV_0 = 1 / sigma_CIV
    return sigma_Lya, sigma_CIV, N_CIV_0

sig_ly, sig_c, N_CIV_0 = CIV(1e5)
print("{:.2e}".format(N_CIV_0))

def tau(sig_c, x):
    return sig_c * x

rr = np.arange(12, 17, 1)
ee = [0,1,2,3,4,5,6,7,8,9,10]


rr_reshaped = rr.reshape(-1, 1)
total_N_c = 10**rr_reshaped * ee / 10
total_N_c = total_N_c.flatten()

C_IV_K = 1548.187e-8  # cm
C_IV_H = 1550.772e-8  # cm
cc = c.to(u.cm / u.s).value
v_th = 11.7673179819719e5  # cm/s

K_gamma = 2.65e+8
H_gamma = 2.64e+8
f_CIV_K = 0.190
f_CIV_H = 0.0952

directory_1 = '~/RT/C_IV/code_tes/vel_test/'
path_sp_1 = f'{directory_1}N_atom000E+00_Vexp000E+00_Vemit500E+01_tauD000E+00_Vran118E+01spec_com.dat'

data_sp_1 = pd.read_csv(path_sp_1, sep='\s+', header=None)
sp_dlam_1 = data_sp_1[0]
yy_g= data_sp_1[1]
# # plt.plot(data_sp_1[0],data_sp_1[1])

nu_space = (cc / (sp_dlam_1 * 1e-8))
del_V_space_g = (sp_dlam_1 *1e-8/ C_IV_K - 1) * cc / 1e5  # km/s

Del_V_H = (C_IV_H/ C_IV_K - 1)* cc / 1e5 

print(Del_V_H)


def gauss_func(F_c,x,x_0,sig):
    xx = np.linspace(x.min(),x.max(),1000)
    ss = (xx-x_0)/sig
    ex = np.zeros(len(xx))
    ex = F_c*np.exp(-ss**2/2)
    return  xx,ex

# plt.plot(del_V_space_g,yy_g)

3.39e+13
500.56195015849033


In [2]:
ee = np.arange(0,610,10)
ee

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540, 550, 560, 570, 580, 590, 600])

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.constants import c
import astropy.units as u

# 데이터 및 상수 설정
rr = np.arange(13, 17, 1)
ee = np.arange(0,610,10)
# ee=[2]

rr_reshaped = rr.reshape(-1, 1)
total_N_c = 10**rr_reshaped * ee / 10
total_N_c = total_N_c.flatten()

C_IV_K = 1548.187e-8  # cm
C_IV_H = 1550.772e-8  # cm
cc = c.to(u.cm / u.s).value
v_th = 11.7673179819719e5  # cm/s

K_gamma = 2.65e+8
H_gamma = 2.64e+8
f_CIV_K = 0.190
f_CIV_H = 0.0952

nu_C_IV_K = cc / C_IV_K  # s-1
nu_C_IV_H = cc / C_IV_H  # s-1

xx_1 = np.zeros(len(rr) * len(ee))
scat_1 = np.zeros(len(rr) * len(ee))

directory_1 = '~/RT/C_IV/code_tes/vel_test/'
n = 0

for ii, num in enumerate(rr):
    for jj, bb in enumerate(ee):
        if bb == 0:
            path_sp_1 = f'{directory_1}N_atom320E+{num}_Vexp{bb}00E+00_Vemit500E+01_tauD000E+00_Vran118E+01spec_com.dat'
        elif bb /100  < 1 :
            path_sp_1 = f'{directory_1}N_atom320E+{num}_Vexp{bb}0E+01_Vemit500E+01_tauD000E+00_Vran118E+01spec_com.dat'
        else:
            path_sp_1 = f'{directory_1}N_atom320E+{num}_Vexp{bb}E+02_Vemit500E+01_tauD000E+00_Vran118E+01spec_com.dat'


        # 데이터 읽기
        try:
            data_sp_1 = pd.read_csv(path_sp_1, sep='\s+', header=None)
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path_sp_1}")
            continue

        sp_dlam_1 = data_sp_1[0].to_numpy()
        sp_1 = data_sp_1[1].to_numpy()

        indices_K_1 = np.where(sp_dlam_1 <= 1550)[0]
        indices_H_1 = np.where(sp_dlam_1 > 1550)[0]

        sp_dlam_K = sp_dlam_1[indices_K_1]
        sp_dlam_H = sp_dlam_1[indices_H_1]

        dlam_k = sp_dlam_K[1]-sp_dlam_K[0]
        dlam_h = sp_dlam_H[1]-sp_dlam_H[0]
        dlam_sp =sp_dlam_1[1] -sp_dlam_1[0]
        # print(dlam_k ,dlam_h,dlam_sp)

        sp_K = sp_1[indices_K_1]
        sp_H = sp_1[indices_H_1]

        if len(sp_K) == 0 or len(sp_H) == 0:
            print(f"데이터 부족: {path_sp_1}")
            continue

        nu_space = (cc / (sp_dlam_1 * 1e-8))
        del_V_space = (nu_C_IV_K / nu_space - 1) * cc / 1e5  # km/s


        indices_K_V = np.where((del_V_space<100)& (del_V_space>-100))[0]
        indices_H_V = np.where((del_V_space<600)& (del_V_space>400))[0]

        sp_V_K = del_V_space[indices_K_V]
        sp_V_H = del_V_space[indices_H_V]

        sp_K_f = sp_1[indices_K_V]
        sp_H_f = sp_1[indices_H_V] 

        KK = np.argmax(sp_K_f)
        HH = np.argmax(sp_H_f)
        K_line_peak_flux = sp_K_f[KK]
        H_line_peak_flux = sp_H_f[HH]



        pp_1 = np.where(sp_1 == K_line_peak_flux)[0]
        pp_1 = pp_1[0] if pp_1.size > 0 else None
        pp_2 = np.where(sp_1 == H_line_peak_flux)[0]
        pp_2 = pp_2[-1] if pp_2.size > 0 else None

        peak_K_V, peak_K_flux = del_V_space[pp_1], sp_1[pp_1]
        peak_H_V, peak_H_flux = del_V_space[pp_2], sp_1[pp_2]


        delta_lam = (sp_dlam_1 - 1548.187 )





        del_V_space_K = (peak_K_V/ sp_V_K - 1) * cc / 1e5  # km/s
        del_V_space_H = (peak_H_V/ nu_space - 1) * cc / 1e5  # km/s


        globals()[f'K_dv_{bb}_{num}'] = del_V_space_K

        globals()[f'Kf_{bb}_{num}'] =sp_K_f

        # print(len(del_V_space_K),len(sp_K_f))


        globals()[f'H_dv_{bb}_{num}'] = del_V_space_H
        globals()[f'Hf_{bb}_{num}'] = sp_H_f

        sep_point = abs(peak_H_V - peak_K_V) / 2
        # print(sep_point)
        indices_K_sp = np.where(del_V_space <= sep_point)[0]
        indices_H_sp = np.where(del_V_space > sep_point)[0]

        int_Vel_K = del_V_space[indices_K_sp]
        int_Vel_H = del_V_space[indices_H_sp]

        int_flux_K = sp_1[indices_K_sp]
        int_flux_H = sp_1[indices_H_sp]

        delta_lam_K = delta_lam[indices_K_sp]
        delta_lam_H = delta_lam[indices_H_sp]
        globals()[f'delta_lam_K_{bb}_{num}'] = delta_lam_K
        globals()[f'delta_lam_H_{bb}_{num}'] = delta_lam_H  

        H_nu = (C_IV_H/C_IV_K -1 )*cc/1e5

        dh = peak_H_V - H_nu  
        globals()[f'I_K_dv_{bb}_{num}'] = int_Vel_K 
        globals()[f'I_K_f_{bb}_{num}'] = int_flux_K

        globals()[f'I_H_dv_{bb}_{num}'] = int_Vel_H  
        globals()[f'I_H_f_{bb}_{num}'] = int_flux_H      